In [2]:
from time import sleep

sleep(0.01)

In [146]:
"""
This cells contains some function to connect coppeliasim api, joint variable initialization & operation
"""

import sim
import time
import sys

import numpy as np

def connect_Client():
    sim.simxFinish(-1)
    clientID = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)         # port-19999

    return clientID

# Now try to retrieve data in a blocking fashion (i.e. a service call):

def Nos_object(clientID = connect_Client()):
    res,objs=sim.simxGetObjects(clientID,sim.sim_handle_all,sim.simx_opmode_blocking)
    if res==sim.simx_return_ok:
        print ('\nNumber of objects in the scene: ',len(objs))
    else:
        print ('\nRemote API function call returned with error code: ',res)
    return len(objs)

# setup control variable

Joints = ['/LBRiiwa7R800/joint',
         '/LBRiiwa7R800/link2_resp/joint',
         '/LBRiiwa7R800/link3_resp/joint', 
         '/LBRiiwa7R800/link4_resp/joint',
         '/LBRiiwa7R800/link5_resp/joint',
         '/LBRiiwa7R800/link6_resp/joint',
         '/LBRiiwa7R800/link7_resp/joint'
         ]


def init_Joint_var(Joints = [], clientID = connect_Client()):
    Joint_Var = []

    for joint in Joints:
        try: err, var = sim.simxGetObjectHandle(clientID, joint, sim.simx_opmode_oneshot_wait)
        except: pass

        if err: print("Failed for Joint", joint, [err, var])
        else: 
            Joint_Var.append(var)
            print("Added for Joint", joint)
    
    print(Joint_Var)

    return Joint_Var

def Get_Joint_Angles(Joints):
    Pos = []
    for Joint in Joint_Var:
        err, pos = sim.simxGetJointPosition(clientID, Joint, sim.simx_opmode_oneshot_wait)
        if err:
            print("Failed for Joint", Joint)
            Pos.append(None)
        else: Pos.append(pos)
    return Pos
            
def Get_Joint_Force(Joints):
    Pos, Max = [], []
    for Joint in Joint_Var:
        err, max = sim.simxGetJointMaxForce(clientID, Joint, sim.simx_opmode_oneshot_wait)
        err, pos = sim.simxGetJointForce(clientID, Joint, sim.simx_opmode_oneshot_wait)
        if err:
            print("Failed for Joint", Joint)
            Pos.append(None)
            Max.append(None)
        else: 
            Pos.append(pos)
            Max.append(max)
    return Pos, Max

def move_robot(q = np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]), Vel = 0.01):
    for i in range(len(Joint_Var)):
        Joint = Joint_Var[i]
        
        err_v = sim.simxSetJointTargetVelocity(clientID, Joint, Vel, sim.simx_opmode_streaming)
        err_p = sim.simxSetJointTargetPosition(clientID, Joint, q[i], sim.simx_opmode_streaming)



clientID = connect_Client()
Joint_Var = init_Joint_var(Joints)

if ~clientID:
    print(clientID, Nos_object())

Get_Joint_Angles(Joints)

Added for Joint /LBRiiwa7R800/joint
Added for Joint /LBRiiwa7R800/link2_resp/joint
Added for Joint /LBRiiwa7R800/link3_resp/joint
Added for Joint /LBRiiwa7R800/link4_resp/joint
Added for Joint /LBRiiwa7R800/link5_resp/joint
Added for Joint /LBRiiwa7R800/link6_resp/joint
Added for Joint /LBRiiwa7R800/link7_resp/joint
[17, 20, 23, 26, 29, 32, 35]

Number of objects in the scene:  80
0 80


[0.11173791438341141,
 -1.8308674097061157,
 -1.1113393156847451e-05,
 -8.272529521491379e-05,
 -0.00017246684001293033,
 0.00022138578060548753,
 1.0969003483296547e-13]

In [112]:
val = 0.001

In [113]:
val = -val
Tau = 65.44
Joint = Joint_Var[1]

#sim.simxSetJointMaxForce(clientID, Joint, 200, sim.simx_opmode_streaming)
sim.simxSetJointForce(clientID, Joint, Tau, sim.simx_opmode_streaming)
sim.simxSetJointTargetVelocity(clientID, Joint, val, sim.simx_opmode_streaming)

val

-0.001

In [120]:
Get_Joint_Force(Joints)

([0.0,
  65.44000244140625,
  -0.8073582053184509,
  51.803955078125,
  -4.747363567352295,
  -24.954853057861328,
  -4.690447807312012],
 [0.0, 65.44000244140625, 100.0, 100.0, 100.0, 38.0, 38.0])

In [148]:
clientID = connect_Client()

In [195]:
Joint = Joint_Var[1]
target = 0
for i in range (1000):
    pos = sim.simxGetJointPosition(clientID, Joint, sim.simx_opmode_oneshot_wait)[1]

    err = target - pos
    print(pos, err)
    if err>0.1: 
        Tau = 50*err*err+0
        dir = 1
    else: 
        Tau = 50*err*err+0
        dir = -1
    if err: 
        sim.simxSetJointTargetVelocity(clientID, Joint, dir, sim.simx_opmode_streaming)
        sim.simxSetJointForce(clientID, Joint, Tau, sim.simx_opmode_streaming)


    #sleep(0.001)

1.8314458131790161 -1.8314458131790161
1.8314722776412964 -1.8314722776412964
1.830859899520874 -1.830859899520874
1.8297418355941772 -1.8297418355941772
1.8281463384628296 -1.8281463384628296
1.8260563611984253 -1.8260563611984253
1.8234670162200928 -1.8234670162200928
1.8203814029693604 -1.8203814029693604


1.8168047666549683 -1.8168047666549683
1.8127446174621582 -1.8127446174621582
1.8081928491592407 -1.8081928491592407
1.8031609058380127 -1.8031609058380127
1.797645926475525 -1.797645926475525
1.7916560173034668 -1.7916560173034668
1.7851969003677368 -1.7851969003677368
1.7782784700393677 -1.7782784700393677
1.7708990573883057 -1.7708990573883057
1.7630712985992432 -1.7630712985992432
1.7547956705093384 -1.7547956705093384
1.746079921722412 -1.746079921722412
1.7369353771209717 -1.7369353771209717
1.7273741960525513 -1.7273741960525513
1.717400312423706 -1.717400312423706
1.7070196866989136 -1.7070196866989136
1.6962462663650513 -1.6962462663650513
1.6850796937942505 -1.6850796937942505
1.6735385656356812 -1.6735385656356812
1.6616301536560059 -1.6616301536560059
1.6493620872497559 -1.6493620872497559
1.6367440223693848 -1.6367440223693848
1.62378990650177 -1.62378990650177
1.6105079650878906 -1.6105079650878906
1.5969096422195435 -1.5969096422195435
1.583001971244812 -1.58300197124481

In [165]:
mat = sim.simxGetJointMatrix(clientID, Joint, sim.simx_opmode_oneshot_wait)[1]
mat

[-0.2596522569656372,
 0.9657021760940552,
 -7.89940415302226e-08,
 3.673216326660622e-07,
 -0.9657021760940552,
 -0.2596522569656372,
 -5.97390865664238e-08,
 1.3243128478279687e-06,
 -7.820114689138791e-08,
 6.077333125631412e-08,
 1.0,
 2.0247523480065865e-06]

In [161]:
Get_Joint_Angles(Joints)

[3.991692210547626e-05,
 0.7853835225105286,
 0.0,
 2.0946145057678223,
 1.037908350554062e-05,
 -1.1444140909588896e-05,
 -4.440892098500626e-16]